In [1]:
import pyspark
from delta import *
from pyspark.sql import SparkSession

builder = SparkSession.builder\
    .appName('spark-bigquery-demo') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.1") \
    .config("spark.sql.warehouse.dir", "gs://lakehouse-prod/spark-warehouse/")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()


# spark.sql("CREATE TABLE IF NOT EXISTS bronze.hihi ( id INT, firstName STRING) USING DELTA;").show()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/g19133035/.ivy2/cache
The jars for the packages stored in: /home/g19133035/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-69df5c98-6001-459d-b931-a768e45de578;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.1 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 314ms :: artifacts dl 8ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.1 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antl

In [2]:
spark.sql("select * from bronze.business;").show()

23/03/19 08:55:04 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

In [4]:
df = spark.sql("describe schema extended silver;")
df.select("*").where("info_name = 'Location'").write.format('json').mode("overwrite").save('gs://lakehouse-prod/test/test.json')

In [24]:
spark.sql("show tables in silver;").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|  silver|     hihi|      false|
+--------+---------+-----------+



In [9]:
spark.sql("CREATE TABLE IF NOT EXISTS silver.hihi ( id INT, firstName STRING) USING DELTA;").show()

23/03/19 08:16:27 WARN org.apache.spark.sql.hive.HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `silver`.`hihi` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
23/03/19 08:16:27 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++

